### 버스 노선 데이터 생성
* 한국교통안전공단_버스노선정보: https://www.data.go.kr/data/15105964/fileData.do
* 국토교통부_(TAGO)_버스노선정보: https://www.data.go.kr/iim/api/selectAPIAcountView.do

1. 전국 버스 노선 정보 데이터 불러오기
2. "국토교통부_(TAGO)_버스노선정보"의 "도시코드 목록 조회" api를 통해 경상남도 도시코드 추출
3. 경상남도 도시코드로 "노선번호목록 조회"를 통해 route data 추출
4. 노선아이디로 "노선정보항목 조회"를 통해 배차간격 데이터 추출
5. 노선 정보를 통해 "노선별경유정류소목록 조회"로 경유지 데이터 추출

In [1]:
import requests
import json

import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

#### 경상남도 도시코드 추출

In [2]:
## 도시코드 목록 조회 api 주소
apiurl_city = "http://apis.data.go.kr/1613000/BusRouteInfoInqireService/getCtyCodeList?serviceKey=J898%2Bzddf%2FpTsegkzSSN3hcgirJ2kkHIbj83qdZjeGTrfWWsvMErAuaux4iEO%2FidrxEJmr99O2IBbzKSk040DQ%3D%3D&_type=json"
response = requests.get(apiurl_city)

## text 형태를 json을 통해 dict 형태로 변형
citycode_result = json.loads(response.text)

## 경상남도 도시코드 추출
city_code = citycode_result["response"]["body"]["items"]["item"]
gyeongn_code = [{code_["cityname"] : code_["citycode"]} for code_ in city_code if str(code_["citycode"])[:2] == "38"]
gyeongn_code = {list(item.keys())[0]: list(item.values())[0] for item in gyeongn_code}

In [3]:
gyeongn_code

{'창원시': 38010,
 '진주시': 38030,
 '통영시': 38050,
 '사천시': 38060,
 '김해시': 38070,
 '밀양시': 38080,
 '거제시': 38090,
 '양산시': 38100,
 '의령군': 38310,
 '함안군': 38320,
 '창녕군': 38330,
 '고성군': 38340,
 '남해군': 38350,
 '하동군': 38360,
 '산청군': 38370,
 '함양군': 38380,
 '거창군': 38390,
 '합천군': 38400}

##### 경상남도 도시코드로 노선번호항목 조회를 통해 route data 추출

In [34]:
def extract_routedata(city_code:int, city_name:str) -> dict:
    """api를 활용해 버스 노선 정보 추출하는 함수

    Args:
        city_code (int): 도시코드 입력
        city_name (str): 도시코드의 시도명 입력

    Returns:
        dict: 도시명, 노선아이디, 노선번호, 버스종류, 기점, 종점, 버스 첫차, 막차
    """
    ## 경상남도 도시코드로 routeId 추출 api
    apiurl_ = f"http://apis.data.go.kr/1613000/BusRouteInfoInqireService/getRouteNoList?serviceKey=J898%2Bzddf%2FpTsegkzSSN3hcgirJ2kkHIbj83qdZjeGTrfWWsvMErAuaux4iEO%2FidrxEJmr99O2IBbzKSk040DQ%3D%3D&pageNo=1&numOfRows=1000&_type=json&cityCode={city_code}"
    response = requests.get(apiurl_)

    ## text 형태를 json을 통해 dict 형태로 변형
    route_result = json.loads(response.text)
    route_ = route_result["response"]["body"]["items"]["item"]
    
    ## route data 추출
    routedata = pd.DataFrame()
    for i, values in enumerate(route_):
        routedata.loc[i, "city_name"] = city_name
        routedata.loc[i, "city_code"] = str(city_code)
        routedata.loc[i, "routeid"] = values["routeid"]
        routedata.loc[i, "routeno"] = str(values["routeno"])
        routedata.loc[i, "routetp"] = values["routetp"]
        routedata.loc[i, "startnodenm"] = values["startnodenm"]    
        routedata.loc[i, "endnodenm"] = values["endnodenm"]

        if "startvehicletime" in values:
            routedata.loc[i, "startvehicletime"] = str(values["startvehicletime"])
        else:
            routedata.loc[i, "startvehicletime"] = None

        if "endvehicletime" in values:
            routedata.loc[i, "endvehicletime"] = str(values["endvehicletime"])
        else:
            routedata.loc[i, "endvehicletime"] = None
                
    return routedata

In [35]:
gyeongn_busroute_bycgg = {cn : extract_routedata(city_code=cc, city_name=cn) for cn, cc in gyeongn_code.items()}
gyeongn_busroute_total = pd.concat([df for key, df in gyeongn_busroute_bycgg.items()], axis=0, ignore_index=True)

#### 노선아이디로 노선정보 조회를 통해 배차간격 데이터 추출

In [51]:
def extract_timedata(city_code, routeid):
    apiurl_ = f"http://apis.data.go.kr/1613000/BusRouteInfoInqireService/getRouteInfoIem?serviceKey=J898%2Bzddf%2FpTsegkzSSN3hcgirJ2kkHIbj83qdZjeGTrfWWsvMErAuaux4iEO%2FidrxEJmr99O2IBbzKSk040DQ%3D%3D&_type=json&cityCode={city_code}&routeId={routeid}"
    response = requests.get(apiurl_)
    time_result = json.loads(response.text)
    time_ = time_result["response"]["body"]["items"]["item"]
    
    if "intervaltime" in time_:
        intervaltime = time_["intervaltime"]
    else:
        intervaltime = None

    if "intervalsattime" in time_:
        intervalsattime = time_["intervalsattime"]
    else:
        intervalsattime = None

    if "intervalsuntime" in time_:
        intervalsuntime = time_["intervalsuntime"]
    else:
        intervalsuntime = None

    return intervaltime, intervalsattime, intervalsuntime

In [55]:
gyeongn_busroute_total[["intervaltime", "intervalsattime", "intervalsuntime"]] = gyeongn_busroute_total.apply(
    lambda row : pd.Series(extract_timedata(row["city_code"], row["routeid"])), axis=1
)

In [58]:
# gyeongn_busroute_total.to_csv("../data/gyeongn_busroute_total.csv", index=False)

#### 노선 정보를 통해 "노선별경유정류소목록 조회"로 경유지 데이터 추출

In [7]:
def extract_nodesdata(
    city_code, 
    routeid:str
    )->dict:
    """도시코드와 노선아이디를 통해 경유지 데이터 추출 

    Args:
        city_code (_type_): 도시코드
        routeid (str): 노선아이디
    Returns:
        dict: 경유지 데이터
    """    
    ## 경유정류소목록 조회 api 주소
    apiurl_nodes = f"http://apis.data.go.kr/1613000/BusRouteInfoInqireService/getRouteAcctoThrghSttnList?serviceKey=J898%2Bzddf%2FpTsegkzSSN3hcgirJ2kkHIbj83qdZjeGTrfWWsvMErAuaux4iEO%2FidrxEJmr99O2IBbzKSk040DQ%3D%3D&pageNo=1&numOfRows=1000&_type=json&cityCode={city_code}&routeId={routeid}"
    response = requests.get(apiurl_nodes)

    ## text 형태를 json을 통해 dict 형태로 변형
    nodes_result = json.loads(response.text)
    nodes = nodes_result["response"]["body"]["items"]["item"]
    
    return nodes  

In [10]:
# nodesdata = {}
# for k, v in tqdm(gyeongn_code.items()):
#     nodesdata[f"{v}"] = {}
#     routeids = gyeongn_busroute_total[gyeongn_busroute_total["city_name"] == k]["routeid"]
#     for id in routeids:
#         nodes = extract_nodesdata(v, id)
#         nodesdata[f"{v}"][f"{id}"] = nodes

100%|██████████| 18/18 [06:32<00:00, 21.80s/it]


In [54]:
# import pickle

# with open("../data/노선경유지데이터.pkl", "wb") as f:
#     pickle.dump(nodesdata, f)